In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras import backend as K

import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
# 
from incpetion_block import *
from keras.models import model_from_json


Using TensorFlow backend.


In [6]:
FRModel = create_model()
print("Total Params:", FRModel.count_params())
FRModel.load_weights('weights/nn4.small2.v1.h5')

Total Params: 3743280
